In [1]:
import pandas as pd

df = pd.read_csv(
    "noticias_unificadas.tsv",
    encoding="utf-8",
    sep="\t",
    dtype={"fecha": "string", "titulo": "string", "contenido": "string", "seccion": "string", "link": "string"},
    quoting=0,
    na_filter=False
)

In [2]:
df.head()

,fecha,titulo,contenido,seccion,link
0,2025-11-09,Jueces rechazan intento de afectación a la ind...,"Desde la ciudad de Tacna, jueces y juezas de t...",Política,https://diariocorreo.pe/politica/jueces-rechaz...
1,2025-11-09,Liga 1: Lo gritan los “Churres” y todo el pueb...,Alianza Atlético le sacó lustre a su clasifica...,Deportes,https://diariocorreo.pe/deportes/alianza-atlet...
2,2025-11-09,Proponen sancionar con hasta 10 años de cárcel...,"La congresista Elizabeth Medina Hermosillo, de...",Política,https://diariocorreo.pe/politica/proponen-sanc...
3,2025-11-09,Este lunes inicia la semana de representación ...,Desde este lunes 10 hasta el viernes 14 de nov...,Política,https://diariocorreo.pe/politica/este-lunes-in...
4,2025-11-09,Selección peruana evalúa reprogramación de par...,La Federación Peruana de Fútbol (FPF) informó ...,Deportes,https://diariocorreo.pe/deportes/seleccion-per...


In [20]:
from utils.utils import clean_text

df["headline_text"] = (df["titulo"].fillna("") + " " + df["titulo"].fillna("") + " " + df["contenido"].fillna("")) + " " + df["seccion"].fillna("")
df["headline_text"] = df["headline_text"].map(clean_text)

df.head()

,fecha,titulo,contenido,seccion,link,headline_text
0,2025-11-09,Jueces rechazan intento de afectación a la ind...,"Desde la ciudad de Tacna, jueces y juezas de t...",Política,https://diariocorreo.pe/politica/jueces-rechaz...,jueces rechazan intento de afectacion a la ind...
1,2025-11-09,Liga 1: Lo gritan los “Churres” y todo el pueb...,Alianza Atlético le sacó lustre a su clasifica...,Deportes,https://diariocorreo.pe/deportes/alianza-atlet...,liga 1: lo gritan los “churres” y todo el pueb...
2,2025-11-09,Proponen sancionar con hasta 10 años de cárcel...,"La congresista Elizabeth Medina Hermosillo, de...",Política,https://diariocorreo.pe/politica/proponen-sanc...,proponen sancionar con hasta 10 anos de carcel...
3,2025-11-09,Este lunes inicia la semana de representación ...,Desde este lunes 10 hasta el viernes 14 de nov...,Política,https://diariocorreo.pe/politica/este-lunes-in...,este lunes inicia la semana de representacion ...
4,2025-11-09,Selección peruana evalúa reprogramación de par...,La Federación Peruana de Fútbol (FPF) informó ...,Deportes,https://diariocorreo.pe/deportes/seleccion-per...,seleccion peruana evalua reprogramacion de par...


In [22]:
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import gensim

STOPWORDS = set(stopwords.words("spanish"))

In [ ]:

STOP_EXTRA = {"dijo","anos","foto","video","puedes","ver","hoy","ayer","manana", "mas", "recomendado", "ser", "dia", "dias", "tambien", "cada", "tras", "soles", "uno", "dos", "tres", "asi", "mil", "ano", "año", "solo", "senalo", "segun", "entre", "millones", "lugar", "puede", "haber", "tener", "sol","precio", "yape", "pai"}
STOPWORDS |= STOP_EXTRA

In [35]:
def lemmatize_stemming(text):
    ps = PorterStemmer()
    return ps.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [36]:
data_text = df[['headline_text']]
data_text['index'] = data_text.index
doc_sample = data_text[data_text['index'] == 4310].values[0][0]
documents = data_text

/var/folders/kg/gsqh5zwj35gdpwht2zn10d3r0000gn/T/ipykernel_99010/179027989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index'] = data_text.index


In [37]:
print(len(documents))
print(documents[:5])

43921
                                       headline_text  index
0  jueces rechazan intento de afectacion a la ind...      0
1  liga 1: lo gritan los “churres” y todo el pueb...      1
2  proponen sancionar con hasta 10 anos de carcel...      2
3  este lunes inicia la semana de representacion ...      3
4  seleccion peruana evalua reprogramacion de par...      4


In [38]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('documento original: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n documento tokenizado y lematizado: ')
print(preprocess(doc_sample))

documento original: 
['funeral', 'de', 'diogo', 'jota', 'y', 'su', 'hermano', 'andre', 'silva', 'se', 'realizara', 'este', 'sabado', 'en', 'gondomar', 'funeral', 'de', 'diogo', 'jota', 'y', 'su', 'hermano', 'andre', 'silva', 'se', 'realizara', 'este', 'sabado', 'en', 'gondomar', 'el', 'funeral', 'para', 'despedir', 'al', 'futbolista', 'portugues', 'diogo', 'jota', '(28)', 'y', 'a', 'su', 'hermano', 'andre', 'silva', '(25)', 'se', 'realizara', 'este', 'sabado', 'a', 'las', '10:00', 'a.m.', '(hora', 'local)', 'en', 'la', 'iglesia', 'matriz', 'de', 'gondomar,', 'segun', 'confirmaron', 'autoridades', 'municipales', 'a', 'medios', 'locales.', 'posteriormente,', 'sus', 'restos', 'seran', 'trasladados', 'al', 'cementerio', 'de', 'la', 'localidad.el', 'viernes,', 'a', 'partir', 'de', 'las', '4:00', 'p.m.,', 'se', 'llevara', 'a', 'cabo', 'el', 'velatorio', 'en', 'la', 'capilla', 'mortuoria', 'de', 'la', 'misma', 'iglesia,', 'donde', 'se', 'espera', 'la', 'presencia', 'de', 'familiares,', 'amigo

In [ ]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0

for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abandono
1 adema
2 afectacion
3 afectarian
4 agravio
5 aprobacion
6 aprobado
7 aprueba
8 aprueban
9 ascenso
10 autonomia


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 83 ("part") appears 1 time.
Word 123 ("aficionado") appears 1 time.
Word 125 ("ahora") appears 1 time.
Word 137 ("atletico") appears 1 time.
Word 171 ("deport") appears 2 time.
Word 174 ("despedir") appears 1 time.
Word 268 ("torneo") appears 1 time.
Word 385 ("autoridad") appears 1 time.
Word 387 ("cabo") appears 1 time.
Word 423 ("llevara") appears 1 time.
Word 424 ("local") appears 2 time.
Word 451 ("segun") appears 1 time.
Word 458 ("viern") appears 1 time.
Word 487 ("europa") appears 1 time.
Word 490 ("futbol") appears 2 time.
Word 491 ("futbolista") appears 1 time.
Word 500 ("miercol") appears 1 time.
Word 701 ("partir") appears 1 time.
Word 774 ("junto") appears 1 time.
Word 785 ("mundo") appears 2 time.
Word 789 ("pasion") appears 1 time.
Word 792 ("presencia") appears 1 time.
Word 800 ("sabado") appears 3 time.
Word 958 ("cuerpo") appears 1 time.
Word 1005 ("figura") appears 1 time.
Word 1071 ("nivel") appears 1 time.
Word 1072 ("noch") appears 1 time.
Word 1090 ("paso") 

In [33]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=9, id2word=dictionary, passes=2, workers=2)

In [32]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.006*"nueva" + 0.006*"cultura" + 0.006*"experiencia" + 0.005*"tambien" + 0.005*"pai" + 0.004*"peruano" + 0.004*"mundo" + 0.004*"lima" + 0.004*"yape" + 0.004*"nacion"
Topic: 1 
Words: 0.006*"libro" + 0.005*"tambien" + 0.005*"cafe" + 0.004*"vizcarra" + 0.004*"cocina" + 0.003*"peruano" + 0.003*"ano" + 0.003*"vino" + 0.003*"sabor" + 0.003*"restaurant"
Topic: 2 
Words: 0.010*"salud" + 0.008*"yape" + 0.005*"promo" + 0.005*"pued" + 0.005*"persona" + 0.004*"caso" + 0.004*"tambien" + 0.004*"segun" + 0.003*"solo" + 0.003*"medico"
Topic: 3 
Words: 0.006*"ano" + 0.006*"vida" + 0.005*"tambien" + 0.005*"ahora" + 0.004*"siempr" + 0.004*"momento" + 0.004*"hijo" + 0.004*"solo" + 0.003*"hace" + 0.003*"nueva"
Topic: 4 
Words: 0.011*"boluart" + 0.009*"ministro" + 0.008*"dina" + 0.008*"fiscal" + 0.007*"caso" + 0.007*"presidenta" + 0.007*"congreso" + 0.007*"politica" + 0.006*"nacion" + 0.006*"ministerio"
Topic: 5 
Words: 0.010*"deport" + 0.009*"alianza" + 0.009*"lima" + 0.007*"equipo" + 0.

## Interpretación y Etiquetado de Tópicos

LDA encuentra tópicos **automáticamente** pero NO les asigna nombres. Los números (0, 1, 2...) son solo índices.

**Tu trabajo**: Interpretar las palabras clave de cada tópico y asignarle un nombre descriptivo.

In [ ]:
import re

def extract_topic_words(lda_model, num_words=10):
    """
    Extrae las palabras principales de cada tópico de forma limpia.
    """
    topics = {}
    for idx, topic in lda_model.print_topics(-1, num_words=num_words):
        # Extraer solo las palabras (sin los pesos)
        words = re.findall(r'"([^"]+)"', topic)
        topics[idx] = words
    return topics

# Extraer palabras clave por tópico
topic_words = extract_topic_words(lda_model, num_words=10)

print("=" * 80)
print("TÓPICOS DESCUBIERTOS POR LDA")
print("=" * 80)

for topic_id, words in topic_words.items():
    print(f"\n🔹 Tópico {topic_id}:")
    print(f"   Palabras clave: {', '.join(words)}")
    print(f"   → Posible tema: [ANALIZAR MANUALMENTE]")

In [ ]:
# Asignar nombres interpretativos a los tópicos (AJUSTAR SEGÚN TUS RESULTADOS)
# Analiza las palabras clave arriba y asigna nombres descriptivos
topic_names = {
    0: "Tema 0 - [Analizar palabras y nombrar]",
    1: "Tema 1 - [Analizar palabras y nombrar]",
    2: "Tema 2 - [Analizar palabras y nombrar]",
    3: "Tema 3 - [Analizar palabras y nombrar]",
    4: "Tema 4 - [Analizar palabras y nombrar]",
    5: "Tema 5 - [Analizar palabras y nombrar]",
    6: "Tema 6 - [Analizar palabras y nombrar]",
    7: "Tema 7 - [Analizar palabras y nombrar]",
    8: "Tema 8 - [Analizar palabras y nombrar]",
    9: "Tema 9 - [Analizar palabras y nombrar]",
}

# Ejemplo de cómo podría verse después de analizar:
# topic_names = {
#     0: "Deportes - Fútbol",
#     1: "Política - Gobierno",
#     2: "Economía - Negocios",
#     3: "Salud - COVID-19",
#     ...
# }

print("\n" + "=" * 80)
print("TÓPICOS CON NOMBRES INTERPRETATIVOS")
print("=" * 80)

for topic_id, words in topic_words.items():
    print(f"\n📌 {topic_names[topic_id]}")
    print(f"   Palabras: {', '.join(words[:7])}")


### 📊 Visualización de Tópicos

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_topics(lda_model, topic_names=None, num_words=8):
    """
    Visualiza los tópicos con sus palabras más importantes.
    """
    topics = lda_model.show_topics(num_topics=-1, num_words=num_words, formatted=False)
    
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    fig.suptitle('Distribución de Palabras por Tópico (LDA)', fontsize=16, fontweight='bold')
    axes = axes.flatten()
    
    for idx, (topic_id, words) in enumerate(topics):
        if idx >= len(axes):
            break
            
        # Extraer palabras y pesos
        word_list = [word for word, _ in words]
        weight_list = [weight for _, weight in words]
        
        # Nombre del tópico
        if topic_names and topic_id in topic_names:
            title = f"Tópico {topic_id}\n{topic_names[topic_id]}"
        else:
            title = f"Tópico {topic_id}"
        
        # Graficar
        axes[idx].barh(range(len(word_list)), weight_list, color='steelblue', alpha=0.7)
        axes[idx].set_yticks(range(len(word_list)))
        axes[idx].set_yticklabels(word_list, fontsize=9)
        axes[idx].set_xlabel('Peso', fontsize=9)
        axes[idx].set_title(title, fontsize=10, fontweight='bold')
        axes[idx].invert_yaxis()
        axes[idx].grid(axis='x', alpha=0.3)
        
        # Valores en barras
        for i, weight in enumerate(weight_list):
            axes[idx].text(weight + 0.001, i, f'{weight:.3f}', 
                          va='center', fontsize=7, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Visualizar (primero sin nombres, luego actualiza topic_names y vuelve a ejecutar)
visualize_topics(lda_model, topic_names=None, num_words=8)

### 🔍 Análisis de documentos por tópico

In [ ]:
# Ver a qué tópico pertenece cada documento
def get_document_topics(lda_model, bow_corpus, df, num_docs=10):
    """
    Muestra ejemplos de documentos y su tópico dominante.
    """
    results = []
    
    for idx, doc_bow in enumerate(bow_corpus[:num_docs]):
        # Obtener distribución de tópicos para este documento
        topic_distribution = lda_model.get_document_topics(doc_bow)
        
        # Encontrar tópico dominante
        if topic_distribution:
            dominant_topic = max(topic_distribution, key=lambda x: x[1])
            topic_id, topic_prob = dominant_topic
            
            results.append({
                'doc_id': idx,
                'titulo': df.iloc[idx]['titulo'][:60] + '...' if len(df.iloc[idx]['titulo']) > 60 else df.iloc[idx]['titulo'],
                'seccion': df.iloc[idx]['seccion'],
                'topic_id': topic_id,
                'topic_prob': topic_prob
            })
    
    return pd.DataFrame(results)

# Analizar primeros documentos
doc_topics_df = get_document_topics(lda_model, bow_corpus, df, num_docs=20)

print("=" * 100)
print("TÓPICO DOMINANTE POR DOCUMENTO")
print("=" * 100)
print(doc_topics_df.to_string(index=False))

print("\n\n📊 Distribución de documentos por tópico:")
print(doc_topics_df['topic_id'].value_counts().sort_index())

In [ ]:
# Comparar tópicos LDA con categorías reales del dataset
topic_vs_category = []

for idx, doc_bow in enumerate(bow_corpus):
    topic_distribution = lda_model.get_document_topics(doc_bow)
    
    if topic_distribution:
        dominant_topic = max(topic_distribution, key=lambda x: x[1])
        topic_id, topic_prob = dominant_topic
        
        topic_vs_category.append({
            'topic_id': topic_id,
            'categoria_real': df.iloc[idx]['seccion']
        })

comparison_df = pd.DataFrame(topic_vs_category)

print("\n" + "=" * 80)
print("COMPARACIÓN: Tópicos LDA vs Categorías Reales")
print("=" * 80)

# Tabla cruzada
crosstab = pd.crosstab(
    comparison_df['categoria_real'], 
    comparison_df['topic_id'], 
    margins=True
)

print(crosstab)

print("\n💡 Interpretación:")
print("   - Cada fila = categoría real del dataset")
print("   - Cada columna = tópico descubierto por LDA")
print("   - Los valores indican cuántos documentos de cada categoría fueron")
print("     asignados a cada tópico")

### 📝 Resumen: Cómo interpretar los resultados

**¿Por qué LDA muestra "Topic 0, 1, 2..." y no nombres?**

- LDA es **no supervisado**: descubre patrones automáticamente
- **No conoce** las categorías reales (Deportes, Política, etc.)
- Los números son solo **identificadores** internos

**Proceso correcto:**

1. ✅ Ejecuta LDA → obtiene tópicos numerados
2. ✅ Analiza las palabras clave de cada tópico
3. ✅ TÚ asignas un nombre interpretativo basándote en las palabras
4. ✅ Compara con las categorías reales para validar

**Ejemplo práctico:**

```
Topic 3: "equipo", "partido", "gol", "jugador", "fútbol"
→ Interpretación: "Deportes - Fútbol"

Topic 7: "presidente", "congreso", "ministro", "gobierno"  
→ Interpretación: "Política - Gobierno"
```